# Test The Informer

In [49]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm
import random 
import os 
import torch 
from torch import nn 
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import time 
import datetime

In [53]:
data = pd.read_csv("public_data/train.csv") 
data.iloc[:,2:]

,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,...,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,5210.0,...,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,5387.0,...,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,1856965.0,1839.0,2055640.0,990.0,2281429.2,990.0,2818.4,19101.0,134359.9,4775.0,...,50730.0,4509.0,282212.3,3001.0,313139.7,3426.0,504242.6,3620.0,283196.9,10940.0
1729,1880095.5,1789.0,1879261.0,1011.0,2074513.0,955.0,1887.1,23095.0,126926.0,5039.0,...,54322.0,4178.0,312214.8,2999.0,362741.0,3357.0,479683.1,3618.0,303779.6,10844.0
1730,1661090.9,1760.0,1709385.7,1075.0,2089081.2,961.0,959.0,22510.0,110357.7,5565.0,...,61213.0,3770.0,327395.8,3065.0,390361.2,3092.0,521493.8,3691.0,313295.7,10636.0
1731,25396.0,3066.0,38222.0,1139.0,18240.0,1056.0,60.0,22333.0,620.0,5227.0,...,144.0,4076.0,285.0,3707.0,2464.0,3252.0,21717.0,3567.0,9734.0,10699.0


In [54]:
submission = pd.read_csv('sample_submission.csv') 

submission.head()

,예측대상일자,배추_가격(원/kg),무_가격(원/kg),양파_가격(원/kg),건고추_가격(원/kg),마늘_가격(원/kg),대파_가격(원/kg),얼갈이배추_가격(원/kg),양배추_가격(원/kg),깻잎_가격(원/kg),...,당근_가격(원/kg),파프리카_가격(원/kg),새송이_가격(원/kg),팽이버섯_가격(원/kg),토마토_가격(원/kg),청상추_가격(원/kg),백다다기_가격(원/kg),애호박_가격(원/kg),캠벨얼리_가격(원/kg),샤인마스캇_가격(원/kg)
0,2020-09-29+1week,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-09-29+2week,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-09-29+4week,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-09-30+1week,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-09-30+2week,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
public_date_list = submission[submission['예측대상일자'].str.contains('2020')]['예측대상일자'].str.split('+').str[0].unique()

In [56]:
public_date_list

array(['2020-09-29', '2020-09-30', '2020-10-01', '2020-10-02',
       '2020-10-03', '2020-10-04', '2020-10-05', '2020-10-06',
       '2020-10-07', '2020-10-08', '2020-10-09', '2020-10-10',
       '2020-10-11', '2020-10-12', '2020-10-13', '2020-10-14',
       '2020-10-15', '2020-10-16', '2020-10-17', '2020-10-18',
       '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22',
       '2020-10-23', '2020-10-24', '2020-10-25', '2020-10-26',
       '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30',
       '2020-10-31', '2020-11-01', '2020-11-02', '2020-11-03',
       '2020-11-04', '2020-11-05'], dtype=object)

# Extract Date Information

In [57]:
date_info = data['date'].values 
weekday_info = data['요일'].values 

In [58]:
week_day_map = {} 
for i,d in enumerate(data['요일'].unique()):
    week_day_map[d] = i  
    
    
weekdays = [] 
for i in range(len(weekday_info)): 
    weekdays.append(week_day_map[weekday_info[i]]) 

In [59]:
date_info[:10], weekdays[:10]

(array(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
        '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
        '2016-01-09', '2016-01-10'], dtype=object),
 [0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [60]:
x_dates = [] 

for i in range(len(date_info)): 
    d = date_info[i].split('-') 
    x_date = [] 
    x_date.append(int(d[0])) 
    x_date.append(int(d[1])) 
    x_date.append(int(d[2])) 
    x_date.append(int(weekdays[i])) 
    x_dates.append(x_date)

In [61]:
x_dates[:1000]

[[2016, 1, 1, 0],
 [2016, 1, 2, 1],
 [2016, 1, 3, 2],
 [2016, 1, 4, 3],
 [2016, 1, 5, 4],
 [2016, 1, 6, 5],
 [2016, 1, 7, 6],
 [2016, 1, 8, 0],
 [2016, 1, 9, 1],
 [2016, 1, 10, 2],
 [2016, 1, 11, 3],
 [2016, 1, 12, 4],
 [2016, 1, 13, 5],
 [2016, 1, 14, 6],
 [2016, 1, 15, 0],
 [2016, 1, 16, 1],
 [2016, 1, 17, 2],
 [2016, 1, 18, 3],
 [2016, 1, 19, 4],
 [2016, 1, 20, 5],
 [2016, 1, 21, 6],
 [2016, 1, 22, 0],
 [2016, 1, 23, 1],
 [2016, 1, 24, 2],
 [2016, 1, 25, 3],
 [2016, 1, 26, 4],
 [2016, 1, 27, 5],
 [2016, 1, 28, 6],
 [2016, 1, 29, 0],
 [2016, 1, 30, 1],
 [2016, 1, 31, 2],
 [2016, 2, 1, 3],
 [2016, 2, 2, 4],
 [2016, 2, 3, 5],
 [2016, 2, 4, 6],
 [2016, 2, 5, 0],
 [2016, 2, 6, 1],
 [2016, 2, 7, 2],
 [2016, 2, 8, 3],
 [2016, 2, 9, 4],
 [2016, 2, 10, 5],
 [2016, 2, 11, 6],
 [2016, 2, 12, 0],
 [2016, 2, 13, 1],
 [2016, 2, 14, 2],
 [2016, 2, 15, 3],
 [2016, 2, 16, 4],
 [2016, 2, 17, 5],
 [2016, 2, 18, 6],
 [2016, 2, 19, 0],
 [2016, 2, 20, 1],
 [2016, 2, 21, 2],
 [2016, 2, 22, 3],
 [2016, 2, 

In [62]:
x_dates = np.asarray(x_dates)

In [63]:
x_dates.shape

(1733, 4)

# Normalization

In [64]:
week_day_map = {} 
for i,d in enumerate(data['요일'].unique()):
    week_day_map[d] = i 
    
data['요일'] = data['요일'].map(week_day_map) 

In [65]:
norm = data.iloc[:,1:].max(0) 
data.iloc[:,1:] = data.iloc[:,1:] / norm

In [66]:
data.head()

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0
1,2016-01-02,0.166667,0.015520,0.0658,0.020797,0.264706,0.024437,0.6405,0.000007,0.064389,...,0.060900,0.511068,0.000197,0.398376,0.023032,0.356152,0.000590,0.08056,0.0,0.0
2,2016-01-03,0.333333,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0
3,2016-01-04,0.500000,0.273068,0.0956,0.440354,0.280882,0.460735,0.6175,0.001690,0.026130,...,0.457792,0.422302,0.227365,0.386475,0.745976,0.297728,0.001814,0.15540,0.0,0.0
4,2016-01-05,0.666667,0.224029,0.0884,0.368802,0.310294,0.416530,0.6065,0.002690,0.025416,...,0.387559,0.383287,0.067041,0.428410,0.278846,0.321334,0.005911,0.11412,0.0,0.0


In [67]:
window_size = 28
future_size = 28    

In [68]:
enc_inputs = [] 
dec_inputs = [] 
targets = [] 
enc_marks = [] 
dec_marks = [] 

for i in tqdm(range(data.shape[0]-window_size-future_size)): 
    x = data.iloc[i:i+window_size, 2:].to_numpy() 
    start_token = data.iloc[i:i+window_size, 3::2].to_numpy() 
    y = data.iloc[i+window_size:i+window_size+future_size, 3::2].to_numpy()  
    y_0 = np.zeros((28,21))
    enc_inputs.append(x) 
    dec_inputs.append(np.concatenate([start_token,y_0]))
    targets.append(y)

    enc_marks.append(x_dates[i:i+window_size]) 
    dec_marks.append(x_dates[i:i+window_size+future_size])
    
enc_inputs = np.asarray(enc_inputs) 
dec_inputs = np.asarray(dec_inputs) 
targets = np.asarray(targets) 
enc_marks = np.asarray(enc_marks) 
dec_marks = np.asarray(dec_marks)

100%|██████████| 1677/1677 [00:01<00:00, 1277.11it/s]


In [69]:
train_enc_inputs, val_enc_inputs, train_dec_inputs, val_dec_inputs = train_test_split(enc_inputs, 
                                                                                      dec_inputs, 
                                                                                      random_state = 888, 
                                                                                      test_size = 0.1)  

train_targets, val_targets, _, _ = train_test_split(targets, 
                                                    dec_inputs, 
                                                    random_state = 888, 
                                                    test_size = 0.1) 


train_enc_marks, val_enc_marks, _, _ = train_test_split(enc_marks, 
                                                        dec_inputs, 
                                                        random_state = 888, 
                                                        test_size = 0.1)

train_dec_marks, val_dec_marks, _, _ = train_test_split(dec_marks, 
                                                        dec_inputs, 
                                                        random_state = 888, 
                                                        test_size = 0.1)


train_enc_inputs.shape, val_enc_inputs.shape, train_dec_inputs.shape, val_dec_inputs.shape, train_targets.shape, val_targets.shape, train_enc_marks.shape, val_enc_marks.shape, train_dec_marks.shape, val_dec_marks.shape


((1509, 28, 42),
 (168, 28, 42),
 (1509, 56, 21),
 (168, 56, 21),
 (1509, 28, 21),
 (168, 28, 21),
 (1509, 28, 4),
 (168, 28, 4),
 (1509, 56, 4),
 (168, 56, 4))

# Define Dataset

In [70]:
class CustomDataset(Dataset): 
    def __init__(self, encoder_input, decoder_input, target_input, encoder_marks, decoder_marks): 
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input 
        self.target_input = target_input  
        self.encoder_marks = encoder_marks 
        self.decoder_marks = decoder_marks 
        
    def __len__(self): 
        return len(self.encoder_input) 
    
    def __getitem__(self, i):  
        return {
            'encoder_input': torch.tensor(self.encoder_input[i], dtype=torch.float32), 
            'decoder_input': torch.tensor(self.decoder_input[i], dtype=torch.float32), 
            'target': torch.tensor(self.target_input[i], dtype=torch.float32),  
            'encoder_marks': torch.tensor(self.encoder_marks[i], dtype=torch.float32),  
            'decoder_marks': torch.tensor(self.decoder_marks[i], dtype=torch.float32) 
        }

In [71]:
BATCH_SIZE = 32

train_dataset = CustomDataset(train_enc_inputs, train_dec_inputs, train_targets, train_enc_marks, train_dec_marks)
val_dataset = CustomDataset(val_enc_inputs, val_dec_inputs, val_targets, val_enc_marks, val_dec_marks)  

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True) 
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = True) 

In [72]:
sample_batch = next(iter(train_dataloader)) 

sample_batch['encoder_input'].shape, sample_batch['decoder_input'].shape, sample_batch['target'].shape

(torch.Size([32, 28, 42]), torch.Size([32, 56, 21]), torch.Size([32, 28, 21]))

In [73]:
sample_batch['encoder_marks'].shape, sample_batch['decoder_marks'].shape 

(torch.Size([32, 28, 4]), torch.Size([32, 56, 4]))

# Define Metric

In [74]:
def my_custom_metric(pred, true): 
    pred = pred[:, [6,13,27]] 
    true = true[:, [6,13,27]] 
    target = torch.where(true!=0) 
    true = true[target] 
    pred = pred[target] 
    score = torch.mean(torch.abs((true-pred)/true)) 
    return score

In [75]:
device = torch.device("cuda:0")

# Model

In [76]:
# add to system path 
import sys
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

In [77]:
from Informer2020.models.model import Informer, InformerStack

model = InformerStack(enc_in = 42, 
                      dec_in = 21, 
                      c_out = 21, 
                      seq_len = 28, 
                      label_len = 28, 
                      out_len = 28, 
                      freq = 'd') 
model.cuda()   


InformerStack(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(42, 512, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TemporalEmbedding(
      (weekday_embed): FixedEmbedding(
        (emb): Embedding(7, 512)
      )
      (day_embed): FixedEmbedding(
        (emb): Embedding(32, 512)
      )
      (month_embed): FixedEmbedding(
        (emb): Embedding(13, 512)
      )
      (year_embed): FixedEmbedding(
        (emb): Embedding(2022, 512)
      )
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(21, 512, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TemporalEmbedding(
      (weekday_embed): FixedEmbedding(
        (emb): Embedding(

In [78]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4) 
criterion = nn.L1Loss() # mae 
custom_metric = my_custom_metric

In [79]:
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [80]:
val_losses = [] 
epochs = 50 
model.zero_grad() 

# for reproducibility 
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


for epoch_i in range(0, epochs): 
    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time() 
    total_loss = 0 
    model.train() 
    
    for step, batch in enumerate(train_dataloader):  
        if step % 10 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))
            
        encoder_input = batch['encoder_input'].to(device) 
        decoder_input = batch['decoder_input'].to(device) 
        target = batch['target'].to(device) 
        enc_marks = batch['encoder_marks'].to(device)  
        dec_marks = batch['decoder_marks'].to(device) 
        
        with torch.cuda.amp.autocast(): 
            output = model(x_enc=encoder_input,
                           x_mark_enc=enc_marks, 
                           x_dec=decoder_input, 
                           x_mark_dec=dec_marks) 
            loss = criterion(output, target) 
            total_loss += loss.item() 
            loss.backward() 
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step() 
            # gradient initialization 
            model.zero_grad() 
    
    avg_train_loss = total_loss / len(train_dataloader) 
    print("")
    print("  Average training loss: {}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        

    # ========================================
    #               Validation
    # ========================================
    
    print("")
    print("Running Validation...")
    t0 = time.time() 
    model.eval() 
    eval_loss = 0 
    for batch in val_dataloader: 
        encoder_input = batch['encoder_input'].to(device) 
        decoder_input = batch['decoder_input'].to(device) 
        target = batch['target'].to(device) 
        enc_marks = batch['encoder_marks'].to(device)  
        dec_marks = batch['decoder_marks'].to(device)   
        
        with torch.no_grad(): 
            outputs = model(x_enc=encoder_input,
                            x_mark_enc=enc_marks, 
                            x_dec=decoder_input, 
                            x_mark_dec=dec_marks) 
            loss = criterion(outputs, target) 
            eval_loss += loss.item()  
        
    avg_val_loss = eval_loss / len(val_dataloader) 
    print("")
    print("  Average validation loss: {}".format(avg_val_loss))
    print("  Validation epoch took: {:}".format(format_time(time.time() - t0))) 
    
    val_losses.append(avg_val_loss) 
    
    if np.min(val_losses) == val_losses[-1]: 
        print("Saving Best Checkpoint....")
        torch.save(model.state_dict(), "INFORMER_" + str(epoch_i + 1)) 

        
        
print("")
print("Training Complete!") 


======== Epoch 1 / 50 ========
Training...
  Batch    10  of     48.    Elapsed: 0:00:01.
  current average loss = 0.3468509539961815
  Batch    20  of     48.    Elapsed: 0:00:02.
  current average loss = 0.25087886974215506
  Batch    30  of     48.    Elapsed: 0:00:03.
  current average loss = 0.21035724033912023
  Batch    40  of     48.    Elapsed: 0:00:04.
  current average loss = 0.18763488288968802

  Average training loss: 0.17581281314293543
  Training epoch took: 0:00:05

Running Validation...

  Average validation loss: 0.14027662575244904
  Validation epoch took: 0:00:00
Saving Best Checkpoint....

======== Epoch 2 / 50 ========
Training...
  Batch    10  of     48.    Elapsed: 0:00:01.
  current average loss = 0.11379319801926613
  Batch    20  of     48.    Elapsed: 0:00:02.
  current average loss = 0.11049469523131847
  Batch    30  of     48.    Elapsed: 0:00:03.
  current average loss = 0.10970126365621885
  Batch    40  of     48.    Elapsed: 0:00:04.
  current aver

  Batch    40  of     48.    Elapsed: 0:00:04.
  current average loss = 0.08339553605765104

  Average training loss: 0.08286292074869077
  Training epoch took: 0:00:04

Running Validation...

  Average validation loss: 0.11673355102539062
  Validation epoch took: 0:00:00

======== Epoch 15 / 50 ========
Training...
  Batch    10  of     48.    Elapsed: 0:00:01.
  current average loss = 0.08392135500907898
  Batch    20  of     48.    Elapsed: 0:00:02.
  current average loss = 0.08207702338695526
  Batch    30  of     48.    Elapsed: 0:00:03.
  current average loss = 0.08191662455598513
  Batch    40  of     48.    Elapsed: 0:00:03.
  current average loss = 0.08131183870136738

  Average training loss: 0.0812752628698945
  Training epoch took: 0:00:04

Running Validation...

  Average validation loss: 0.10015162204702695
  Validation epoch took: 0:00:00
Saving Best Checkpoint....

======== Epoch 16 / 50 ========
Training...
  Batch    10  of     48.    Elapsed: 0:00:01.
  current avera

  Batch    30  of     48.    Elapsed: 0:00:03.
  current average loss = 0.07910440390308698
  Batch    40  of     48.    Elapsed: 0:00:04.
  current average loss = 0.0788056120276451

  Average training loss: 0.07859769851590197
  Training epoch took: 0:00:04

Running Validation...

  Average validation loss: 0.11327376713355382
  Validation epoch took: 0:00:00

======== Epoch 29 / 50 ========
Training...
  Batch    10  of     48.    Elapsed: 0:00:01.
  current average loss = nan
  Batch    20  of     48.    Elapsed: 0:00:02.
  current average loss = nan
  Batch    30  of     48.    Elapsed: 0:00:03.
  current average loss = nan
  Batch    40  of     48.    Elapsed: 0:00:04.
  current average loss = nan

  Average training loss: nan
  Training epoch took: 0:00:04

Running Validation...

  Average validation loss: nan
  Validation epoch took: 0:00:00

======== Epoch 30 / 50 ========
Training...
  Batch    10  of     48.    Elapsed: 0:00:01.
  current average loss = nan
  Batch    20  of

  Batch    10  of     48.    Elapsed: 0:00:01.
  current average loss = nan
  Batch    20  of     48.    Elapsed: 0:00:02.
  current average loss = nan
  Batch    30  of     48.    Elapsed: 0:00:03.
  current average loss = nan
  Batch    40  of     48.    Elapsed: 0:00:04.
  current average loss = nan

  Average training loss: nan
  Training epoch took: 0:00:04

Running Validation...

  Average validation loss: nan
  Validation epoch took: 0:00:00

======== Epoch 46 / 50 ========
Training...
  Batch    10  of     48.    Elapsed: 0:00:01.
  current average loss = nan
  Batch    20  of     48.    Elapsed: 0:00:02.
  current average loss = nan
  Batch    30  of     48.    Elapsed: 0:00:03.
  current average loss = nan
  Batch    40  of     48.    Elapsed: 0:00:03.
  current average loss = nan

  Average training loss: nan
  Training epoch took: 0:00:04

Running Validation...

  Average validation loss: nan
  Validation epoch took: 0:00:00

======== Epoch 47 / 50 ========
Training...
  Ba

# Inference

In [81]:
checkpoint = torch.load('INFORMER_15') 
test_model = InformerStack(enc_in = 42, 
                           dec_in = 21, 
                           c_out = 21, 
                           seq_len = 28, 
                           label_len = 28, 
                           out_len = 28, 
                           freq = 'd') 
test_model.load_state_dict(checkpoint) 
test_model.eval() 
test_model.cuda()


InformerStack(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(42, 512, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TemporalEmbedding(
      (weekday_embed): FixedEmbedding(
        (emb): Embedding(7, 512)
      )
      (day_embed): FixedEmbedding(
        (emb): Embedding(32, 512)
      )
      (month_embed): FixedEmbedding(
        (emb): Embedding(13, 512)
      )
      (year_embed): FixedEmbedding(
        (emb): Embedding(2022, 512)
      )
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(21, 512, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TemporalEmbedding(
      (weekday_embed): FixedEmbedding(
        (emb): Embedding(

In [82]:
submission = pd.read_csv('sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2020')]['예측대상일자'].str.split('+').str[0].unique()

In [83]:
# use the same weekday map as the training set 
week_day_map 

{'금요일': 0, '토요일': 1, '일요일': 2, '월요일': 3, '화요일': 4, '수요일': 5, '목요일': 6}

In [84]:
# a naive version for testing 
def get_decoder_marks(dates):
    last_date = dates[-1] 
    cur_date = last_date 
    next_dates = [] 
    for i in range(28): 
        next_day = cur_date[2] + 1  
        next_month = cur_date[1]
        next_year = cur_date[0] 

        if next_day == 31 and cur_date[1] in [4,6,9,11]:  
            next_day = 1 
            next_month = cur_date[1] + 1   
        elif next_day == 32 and cur_date[1] in [1,3,5,7,8,10,12]:  
            next_day = 1 
            next_month = cur_date[1] + 1 
            
        next_weekday = cur_date[3] + 1 
        if next_weekday > 6: 
            next_weekday = 0 
            
        final_date = [next_year, next_month, next_day, next_weekday] 
        cur_date = final_date
        
        next_dates.append(final_date)  
    
    return np.asarray(next_dates)  
            

In [92]:
predictions = [] 
torch_norm = torch.tensor(norm.to_numpy()[2::2]) 

test_model.eval() 

# change to eval mode 
for date in tqdm(public_date_list):
    test_df = pd.read_csv(f'./public_data/test_files/test_{date}.csv')
    data = pd.read_csv('./public_data/train.csv')
    data = pd.concat([data, test_df]).iloc[-window_size:]
    
    date_info = data['date'].values 
    weekday_info = data['요일'].values
    data['요일'] = data['요일'].map(week_day_map) 
        
    weekdays = [] 
    for i in range(len(weekday_info)): 
        weekdays.append(week_day_map[weekday_info[i]]) 
    
    x_test_dates = [] 

    for i in range(len(date_info)): 
        d = date_info[i].split('-') 
        x_date = [] 
        x_date.append(int(d[0])) 
        x_date.append(int(d[1])) 
        x_date.append(int(d[2])) 
        x_date.append(int(weekdays[i])) 
        x_test_dates.append(x_date) 
        
        
    data = data.iloc[:,1:] / norm  

    
    # get encoder_input, decoder_input, encoder_marks, decoder_marks
    test_encoder_inputs = data.iloc[:, 1:].to_numpy() 
    test_decoder_inputs = data.iloc[:, 2::2].to_numpy() 
    target_tokens = np.zeros((28,21)) 
    test_decoder_inputs = np.concatenate([test_decoder_inputs,target_tokens])
    
    test_encoder_marks = np.asarray(x_test_dates) 
    test_decoder_marks = get_decoder_marks(x_test_dates) 
    test_decoder_marks = np.concatenate([x_test_dates, test_decoder_marks]) 
    
    
    # convert to torch tensors and reshape 
    test_encoder_inputs = torch.tensor(test_encoder_inputs, dtype=torch.float32)
    test_encoder_inputs = torch.reshape(test_encoder_inputs, (-1,test_encoder_inputs.shape[0], test_encoder_inputs.shape[1])) 
    test_encoder_inputs = test_encoder_inputs.to(device)
    
    test_decoder_inputs = torch.tensor(test_decoder_inputs, dtype=torch.float32)
    test_decoder_inputs = torch.reshape(test_decoder_inputs, (-1,test_decoder_inputs.shape[0], test_decoder_inputs.shape[1]))
    test_decoder_inputs = test_decoder_inputs.to(device) 
    
    test_encoder_marks = torch.tensor(test_encoder_marks, dtype=torch.float32)
    test_encoder_marks = torch.reshape(test_encoder_marks, (-1,test_encoder_marks.shape[0],test_encoder_marks.shape[1]))
    test_encoder_marks = test_encoder_marks.to(device) 
    
    test_decoder_marks = torch.tensor(test_decoder_marks, dtype=torch.float32) 
    test_decoder_marks = torch.reshape(test_decoder_marks, (-1,test_decoder_marks.shape[0],test_decoder_marks.shape[1]))
    test_decoder_marks = test_decoder_marks.to(device)  
    
        
    
    with torch.no_grad(): 
        outputs = test_model(x_enc=test_encoder_inputs,
                             x_mark_enc=test_encoder_marks, 
                             x_dec=test_decoder_inputs, 
                             x_mark_dec=test_decoder_marks) 
    
    outputs = outputs.cpu()

    outputs = outputs*torch_norm  
        
    
    idx = submission[submission['예측대상일자'].str.contains(date)].index
    submission.loc[idx, '배추_가격(원/kg)':] = outputs[0,[6,13,27]].numpy()   
    
    

100%|██████████| 38/38 [00:01<00:00, 22.29it/s]


In [93]:
submission

,예측대상일자,배추_가격(원/kg),무_가격(원/kg),양파_가격(원/kg),건고추_가격(원/kg),마늘_가격(원/kg),대파_가격(원/kg),얼갈이배추_가격(원/kg),양배추_가격(원/kg),깻잎_가격(원/kg),...,당근_가격(원/kg),파프리카_가격(원/kg),새송이_가격(원/kg),팽이버섯_가격(원/kg),토마토_가격(원/kg),청상추_가격(원/kg),백다다기_가격(원/kg),애호박_가격(원/kg),캠벨얼리_가격(원/kg),샤인마스캇_가격(원/kg)
0,2020-09-29+1week,847.736299,622.958150,763.107955,17767.369491,4726.961725,1937.300964,874.740040,790.082339,2827.664690,...,1543.306187,3657.336922,1653.562665,1558.366600,1978.029460,2268.147852,1975.494076,1098.205279,2117.845975,7496.036291
1,2020-09-29+2week,872.360468,748.475204,754.129648,15041.441629,4781.794223,2094.085039,942.172809,970.965916,4476.921110,...,1677.199602,4038.487540,1978.420615,1855.822767,2814.846479,3660.647615,2269.150782,1537.469433,2350.755967,3933.650255
2,2020-09-29+4week,1012.851000,1001.380391,813.972235,18219.587242,5275.259938,2181.421726,1390.371976,1216.404835,8324.011843,...,1933.192983,5436.511889,2598.682880,2344.333830,4154.447987,5233.349069,2469.133577,2736.994448,3101.043589,1223.930791
3,2020-09-30+1week,809.706300,609.735913,773.450613,16867.078315,4738.021598,1889.236635,830.138902,821.702757,2283.292237,...,1511.206403,3355.762516,1605.403304,1519.446560,1881.702787,1525.904181,1857.848692,907.637639,2023.351006,7934.085131
4,2020-09-30+2week,891.161263,729.734731,767.378688,13982.647804,4782.371974,2049.027422,934.258050,1011.369930,4413.143608,...,1654.286608,3715.861926,1980.861783,1870.338272,2700.905111,3171.180625,2171.252013,1438.130309,2161.055803,3901.496530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2021-11-03+2week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
224,2021-11-03+4week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
225,2021-11-04+1week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
226,2021-11-04+2week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [94]:
submission.to_csv("Informer_price_volume.csv",index=False)